In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain.prompts import PromptTemplate
from langchain.output_parsers import (
    CommaSeparatedListOutputParser,
    StructuredOutputParser,ResponseSchema
)
from langchain.llms import OpenAI

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
llm=OpenAI(model="gpt-3.5-turbo-instruct")

c:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [4]:
outputParser=CommaSeparatedListOutputParser()

In [5]:
formatInstructions=outputParser.get_format_instructions()
print(formatInstructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [6]:
template=""" 
Provide 5 examples of {format_instructions}.\n{query}
"""

In [7]:
prompt=PromptTemplate(
    template=template,
    input_variables=["query"],
    partial_variables={"format_instructions":formatInstructions}
)

In [13]:
currency="Currencies"
response=llm.invoke(
    input=prompt.format(
        query=currency
    )
                    )

In [14]:
print(response)

1. USD 
2. EUR 
3. JPY 
4. GBP 
5. CAD


In [15]:
print((prompt | llm).invoke(input={"query":currency}))

1. USD
2. EUR
3. JPY
4. GBP
5. AUD


### JsonFormat

In [44]:
responseSchemas=[
    ResponseSchema(
        name="currency",
        description="Answer to the User's Question"
    ),
    ResponseSchema(
        name="abbreviation",
        description="What's the abbreviation of that currency"
    )
]

In [45]:
responseSchemas

[ResponseSchema(name='currency', description="Answer to the User's Question", type='string'),
 ResponseSchema(name='abbreviation', description="What's the abbreviation of that currency", type='string')]

In [46]:
outputParser=StructuredOutputParser.from_response_schemas(
    response_schemas=responseSchemas
    )

In [47]:
outputParser

StructuredOutputParser(response_schemas=[ResponseSchema(name='currency', description="Answer to the User's Question", type='string'), ResponseSchema(name='abbreviation', description="What's the abbreviation of that currency", type='string')])

In [48]:
formatInstructions=outputParser.get_format_instructions()
print(formatInstructions)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // Answer to the User's Question
	"abbreviation": string  // What's the abbreviation of that currency
}
```


In [49]:
template=""" 
Answer the User's Question as best as possible.\n{formatInstructions}\n{query}
"""

In [50]:
prompt=PromptTemplate(
    template=template,
    input_variables=["query"],
    partial_variables={"formatInstructions":formatInstructions}
)

In [51]:
print(prompt)

input_variables=['query'] partial_variables={'formatInstructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"currency": string  // Answer to the User\'s Question\n\t"abbreviation": string  // What\'s the abbreviation of that currency\n}\n```'} template=" \nAnswer the User's Question as best as possible.\n{formatInstructions}\n{query}\n"


In [52]:
currency="Singaporean Dollar"
response=llm.invoke(
    input=prompt.format(
        query=currency
    )
                    )

In [53]:
print(response)

```json
{
	"currency": "Singaporean Dollar",
	"abbreviation": "SGD"
}
```

